In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
     

In [5]:
# !wget https://www.comp.nus.edu.sg/~nlp/conll14st/conll14st-test-data.tar.gz -P /content/

# !tar -xvzf '/content/conll14st-test-data.tar.gz' -C '/content/drive/MyDrive/GrammarCorrection/Data/'
 

--2022-12-09 10:58:00--  https://www.comp.nus.edu.sg/~nlp/conll14st/conll14st-test-data.tar.gz
Resolving www.comp.nus.edu.sg (www.comp.nus.edu.sg)... 45.60.31.225
Connecting to www.comp.nus.edu.sg (www.comp.nus.edu.sg)|45.60.31.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 643482 (628K) [application/x-gzip]
Saving to: ‘/content/conll14st-test-data.tar.gz’

conll14st-test-data 100%[===================>] 628.40K   151KB/s    in 4.2s    

2022-12-09 10:58:05 (151 KB/s) - ‘/content/conll14st-test-data.tar.gz’ saved [643482/643482]

conll14st-test-data/
conll14st-test-data/scripts/
conll14st-test-data/scripts/parser_feature.py
conll14st-test-data/scripts/preprocess.py
conll14st-test-data/scripts/nuclesgmlparser.py
conll14st-test-data/scripts/README
conll14st-test-data/scripts/nucle_doc.py
conll14st-test-data/scripts/preprocesscombine.py
conll14st-test-data/scripts/preprocesswithalt.py
conll14st-test-data/scripts/iparser.py
conll14st-test-data/noalt/
conll14st

In [7]:
def process_m2file(m2_file, out_file, id=0):

  """
  This function used to convert .m2 file to .txt file. 
  This functions return dataframe with correct, incorrect, total_edits 

  Arguments:
  m2_file: path to .m2 file
  out: path to .txt file
  id: Annotator id

  converts specified annotators text from erroneous to non-erroneous
  """
  
  m2 = open(m2_file).read().strip().split("\n\n")
  out = open(out_file, "w")
  # Do not apply edits with these error types
  skip = {"noop", "UNK", "Um"}
  total_edits = []
  
  for sent in m2:
    sent = sent.split("\n")
    cor_sent = sent[0].split()[1:] # Ignore "S "
    edits = sent[1:]
    offset = 0
    cur_edit = 0
    for edit in edits:
      edit = edit.split("|||")
      if edit[1] in skip: continue # Ignore certain edits
      coder = int(edit[-1])
      if coder != id: continue # Ignore other coders
      span = edit[0].split()[1:] # Ignore "A "
      start = int(span[0])
      end = int(span[1])
      cor = edit[2].split()
      cor_sent[start+offset:end+offset] = cor
      cur_edit+=1
      offset = offset-(end-start)+len(cor)
    total_edits.append(cur_edit)
    out.write(" ".join(cor_sent)+"\n")
  out.close()

  file1 = open(m2_file,"r")
  s1 = file1.read()

  each_sent = s1.split("\n\n")

  incorrect = []
  for i in range(len(each_sent)):
      temp = each_sent[i].split("\n")
      temp = temp[0]
      temp = temp.split(" ")
      temp = temp[1:]# ignore S
      temp = ' '.join(temp)
      incorrect.append(temp)
      
  #preprocessing correct sentences

  file2 = open(out_file, "r")
  s2 = file2.read()

  correct = s2.split("\n")

  print("Preparing dataframe")
  df = pd.DataFrame()
  df["correct"] = correct
  df["incorrect"] = incorrect
  total_edits.append(0)
  df['total_edits'] = total_edits

  return df

In [10]:
data_id0 = process_m2file(m2_file='/content/drive/My Drive/GrammarCorrection/Data/conll14st-test-data/noalt/official-2014.combined.m2',
                      out_file='/content/drive/My Drive/GrammarCorrection/Data/conll14st-test-data/noalt/official-2014.combined.m2_id0_corrected.txt',
                      id=0)

Preparing dataframe


In [11]:
data_id0.shape

(1313, 3)

In [13]:
#save dataframe
data_id0.to_csv('/content/drive/MyDrive/GrammarCorrection/Data/data_collection/conll14_combined_id0.csv', index=False)

#Lang8 data set

In [15]:
!wget --header="Host: doc-0g-7s-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Cookie: AUTH_bethcp9rn39ka2kmrd545br566r951gq_nonce=qujc9fn88q1bg" --header="Connection: keep-alive" "https://doc-0g-7s-docs.googleusercontent.com/docs/securesc/22md60cfnqgvnvkdfr99parer0vfbk67/chk935ml30qevkgl9topbpkcjb36nf07/1670583900000/15187745062133414559/02427598508910905988/148M_4LvHyb0J_sNWxrFSYjv3lO5ACigK?e=download&ax=AEKYgySSai5T4Z4ddToDIaHGobOxv9zTfktZdR93SiV6WbUKYquUjFRBXmklJF_lODAxoVuxowrzgRjFnfLDppfQ7EoGAS7PmlxanqkZOjcZL2ESrVglh2XZ5wAZZv48sQ5SigQgFxQ9V78UaIqUbzityVZydPoxX2tr58AXo5I0h5CMAbZaJ8I9vTdC35PUjANI8Po0WIkh4QQFWvwNhYyEoAD3IeYWWV73jAKc-_n4KV-F7JlVlQlypzx88ZJo5H7VOYMxAfxYnaSZXM7w0RWRxoukYLdgzMgpwC5nJxUeCrCTyXEHHBpbJXMsNV9c11JoDd7kj2WaSZUGhjkCsxQ8BdLU7-Fct_VfsnOLuIcWhgKmR0KTIjw9dsVtwzP-BuJjUpTzdj3qy_AjQN__zFqJ6I3yiIm7F8YJ7qh06SElUxc8rJtZfYD8nLSye5OedJw6F6b-WHamQQCe1NmWrCATJHsgwR9BLz416iYmeAm4leBNpRauaPFN-dxMPldG2zCH88-Xa7YN7spdCfNH6U10XHjy_l5DzmUpXRPjg71fDavjBDSsdl72ISZEGOchsR5YbkVCegFh1ain0GvT4Xz3MHetqVbJ5Io28F-ckCpV3j6HUSZx3ysgXBX1gE0FYBZEWzd6_ZVviDL170sUOg6jzvHPb0IPLX6lcZbly4h2Arkyn7YFxpIQV3d6AzN2Wbb5af8Hdq08cX1pRKsEWmbd1ihJuqa8J8xW5uLwDWeEkGW-3Vg_COGcTY7tpj7U7PAC22Z_dn7Dsr-1zHngKYWGyjE15A-AB2vRyrEBAOlF-zHPJpIz7_9osDCOFsw--k3yjaLHCdGJuTGfyoDDAGLDk_gc8k20L6-oDIhEEFkP_6w2Fk1scALuvcEV7YxhN8Q4AlxUZfGg0kby1q5PA0RhaciVtAP1eetau280IN8&uuid=64d88c8d-dcf7-456c-8a94-73e08d81fc12&authuser=0&nonce=qujc9fn88q1bg&user=02427598508910905988&hash=gbopnkoijageg136qd19vnetjttih9ac" -c -O 'lang8.bea19.tar.gz'

--2022-12-09 11:05:44--  https://doc-0g-7s-docs.googleusercontent.com/docs/securesc/22md60cfnqgvnvkdfr99parer0vfbk67/chk935ml30qevkgl9topbpkcjb36nf07/1670583900000/15187745062133414559/02427598508910905988/148M_4LvHyb0J_sNWxrFSYjv3lO5ACigK?e=download&ax=AEKYgySSai5T4Z4ddToDIaHGobOxv9zTfktZdR93SiV6WbUKYquUjFRBXmklJF_lODAxoVuxowrzgRjFnfLDppfQ7EoGAS7PmlxanqkZOjcZL2ESrVglh2XZ5wAZZv48sQ5SigQgFxQ9V78UaIqUbzityVZydPoxX2tr58AXo5I0h5CMAbZaJ8I9vTdC35PUjANI8Po0WIkh4QQFWvwNhYyEoAD3IeYWWV73jAKc-_n4KV-F7JlVlQlypzx88ZJo5H7VOYMxAfxYnaSZXM7w0RWRxoukYLdgzMgpwC5nJxUeCrCTyXEHHBpbJXMsNV9c11JoDd7kj2WaSZUGhjkCsxQ8BdLU7-Fct_VfsnOLuIcWhgKmR0KTIjw9dsVtwzP-BuJjUpTzdj3qy_AjQN__zFqJ6I3yiIm7F8YJ7qh06SElUxc8rJtZfYD8nLSye5OedJw6F6b-WHamQQCe1NmWrCATJHsgwR9BLz416iYmeAm4leBNpRauaPFN-dxMPldG2zCH88-Xa7YN7spdCfNH6U10XHjy_l5DzmUpXRPjg71fDavjBDSsdl72ISZEGOchsR5YbkVCegFh1ain0GvT4Xz3MHetqVbJ5Io28F-ckCpV3j6HUSZx3ysgXBX1gE0FYBZEWzd6_ZVviDL170sUOg6jzvHPb0IPLX6lcZbly4h2Arkyn7YFxpIQV3d6AzN2Wbb5af8Hdq08cX1pRKsEWmbd1ihJuqa8J8xW5uLwDW

In [17]:
!tar -xvzf '/content/lang8.bea19.tar.gz' -C '/content/drive/My Drive/GrammarCorrection/Data/data_collection/'
     

lang8.train.auto.bea19.m2
lang8_to_m2.py
readme.txt


In [18]:
data_lang8_id0 = process_m2file(m2_file='/content/drive/MyDrive/GrammarCorrection/Data/data_collection/lang8.train.auto.bea19.m2',
                            out_file='/content/drive/MyDrive/GrammarCorrection/Data/data_collection/lang8.train.auto.bea19.m2_id0_corrected.txt',
                            id=0)
data_lang8_id0.shape

Preparing dataframe


(1037562, 3)

In [19]:
data_lang8_id0.to_csv('/content/drive/MyDrive/GrammarCorrection/Data/data_collection/lang8_anno_id_0.csv', index=False)
     

In [20]:
data_lang8_id1 = process_m2file(m2_file='/content/drive/MyDrive/GrammarCorrection/Data/data_collection/lang8.train.auto.bea19.m2',
                            out_file='/content/drive/MyDrive/GrammarCorrection/Data/data_collection/lang8.train.auto.bea19.m2_id1_corrected.txt',
                            id=1)
     

Preparing dataframe


In [21]:
data_lang8_id1.to_csv('/content/drive/MyDrive/GrammarCorrection/Data/data_collection/lang8_anno_id1.csv', index=False)
     